In [1]:
#!pip install pyspark pandas boto3 pyarrow --upgrade

In [2]:
import sys
import os
import pandas as pd
sys.path.append('/home/darcy/kensho/') # add local library to path
os.environ['SPARK_VERSION'] = '3.3' # required for deequ
import kensho

In [3]:

engine = kensho.ValidationEngine()

In [4]:
from pyspark.sql import SparkSession

try:
    spark.stop()
except:
    print('no session yet')
    
# Create a SparkSession
spark = SparkSession.builder \
    .appName("ValidationEngine") \
    .config("spark.jars.packages", "com.amazon.deequ:deequ:2.0.4-spark-3.3") \
    .getOrCreate()

no session yet
:: loading settings :: url = jar:file:/home/darcy/spark3.3.2/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/darcy/.ivy2/cache
The jars for the packages stored in: /home/darcy/.ivy2/jars
com.amazon.deequ#deequ added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2900da4d-70e4-4951-a3b7-d7ff68cacaa4;1.0
	confs: [default]
	found com.amazon.deequ#deequ;2.0.4-spark-3.3 in central
	found org.scala-lang#scala-reflect;2.12.10 in central
	found org.scalanlp#breeze_2.12;0.13.2 in central
	found org.scalanlp#breeze-macros_2.12;0.13.2 in central
	found com.github.fommil.netlib#core;1.1.2 in central
	found net.sourceforge.f2j#arpack_combined_all;0.1 in central
	found net.sf.opencsv#opencsv;2.3 in central
	found com.github.rwl#jtransforms;2.4.0 in central
	found junit#junit;4.8.2 in central
	found org.apache.commons#commons-math3;3.2 in central
	found org.spire-math#spire_2.12;0.13.0 in central
	found org.spire-math#spire-macros_2.12;0.13.0 in central
	found org.typelevel#machinist_2.12;0.6.1 in central
	found com.chuusai#shapeless_2.12;2.

23/08/23 14:22:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
# provide spark session to engine
engine.set_spark(spark)

In [6]:
# schema for the check we are running
# [rule_id, rule_type, comparison, threshold, alert_level]

json_config = {
    "inputs": {
        "example": {
        }
    },
    "resolver_map": {
        "example": "its_a_csv_right_here"
    },
    "loader_map": {
        "example": "spark_read_csv"
    },
    "validation_object_loader_map": {
        "example": "its_already_the_object"
    },
    "validation_objects": {
        "example": {
            "inputs": ["example"]
        }
    },
    "validations": {
        "example": [["rule-001", "has_row_count", "ge", 4, "warning"]]
    }
}
app_parameters = {
    "s3_prefix": None, 
    "default_bucket": None,
    "project_path": None,
    "validation_dataset_name": "example",
    "validation_dataset_bucket": None

}
run_parameters = {
    "pipeline_execution_id": None,
    "effective_extract_dt": '20230822'
}

# load all required config to engine for the dataset we want to validate
engine.load_from(json_config, app_parameters, run_parameters)

In [7]:
# run the full validations using config provided
engine.run()

running entire engine
current execution ID: manual-execution-1-20230824002257
loading input object: example
example.csv


/home/darcy/my-python3-env/lib/python3.10/site-packages/pyspark/pandas/__init__.py:49: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


generated input objects
dict_keys(['example'])
loading validation object: example
generated validation objects
dict_keys(['example'])
Python Callback server started!


/home/darcy/my-python3-env/lib/python3.10/site-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
/home/darcy/my-python3-env/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


test


/home/darcy/my-python3-env/lib/python3.10/site-packages/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/home/darcy/my-python3-env/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/darcy/my-python3-env/lib/python3.10/site-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: `to_pandas` loads all data into the driver's memory. It should only be used if the resulting pandas DataFrame is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
/home/darcy/spark3.3.2/python/lib/pyspark.zip/pyspark/pandas/__init__.py:49: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarro

Was unable to order verbose results columns
generated validation results
dict_keys(['example'])
finished
Number of executions is now 1


/home/darcy/spark3.3.2/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.


In [8]:
# show verbose results dataframe
engine.verbose_results

,rule_id,check_level,check_status,constraint_status,validation_metric_value,rule_type,operator,threshold,alert_level,validation_dataset
0,rule-001,NaN,NaN,Success,5,has_row_count,ge,4.0,warning,example


In [9]:
import numpy as np
# Generate a pandas DataFrame
pdf = pd.DataFrame(np.random.rand(100, 3))

# Create a Spark DataFrame from a pandas DataFrame using Arrow
df = spark.createDataFrame(pdf)

# Convert the Spark DataFrame back to a pandas DataFrame using Arrow
result_pdf = df.select("*").toPandas()

/home/darcy/my-python3-env/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/darcy/my-python3-env/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [11]:
# !pip install pandas==1.5.3
# pyspark==3.3.2
!pip list

Package           Version
----------------- -----------
asttokens         2.2.1
backcall          0.2.0
boto3             1.28.32
botocore          1.31.32
comm              0.1.4
debugpy           1.6.7.post1
decorator         5.1.1
executing         1.2.0
ipykernel         6.25.1
ipython           8.14.0
jedi              0.19.0
jmespath          1.0.1
jupyter_client    8.3.0
jupyter_core      5.3.1
matplotlib-inline 0.1.6
nest-asyncio      1.5.7
numpy             1.25.2
packaging         23.1
pandas            1.5.3
parso             0.8.3
pexpect           4.8.0
pickleshare       0.7.5
pip               22.0.2
platformdirs      3.10.0
prompt-toolkit    3.0.39
psutil            5.9.5
ptyprocess        0.7.0
pure-eval         0.2.2
py4j              0.10.9.5
pyarrow           13.0.0
pydeequ           1.1.0
Pygments          2.16.1
pyspark           3.3.2
python-dateutil   2.8.2
pytz              2023.3
pyzmq             25.1.1
s3transfer        0.6.2
setuptools        59.6.0
six     